In [ ]:
# prepare dependencies

import numpy as np
from utils import load_surf, load_config, butterworth_highpass, zscore, make_delayed, get_envelope, lanczosinterp2D, get_logger
from ridge import ridge_cv
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import nibabel as nib
import os
import time
from nilearn import plotting, surface, datasets
from scipy.io import wavfile

cfg = load_config()
DATADIR = cfg['DATA_DIR']
STIMDIR = cfg['STIM_DIR']
FIGDIR = cfg['FIG_DIR']

logging = get_logger()

subject = 'sub-007'
session = 'ses-01'

tr = 1.5

fsaverage = datasets.fetch_surf_fsaverage('fsaverage6')

In [ ]:
# story audio envelope encoding model
task = 'treasureisland'
hemi = 'L'
surf_file = os.path.join(DATADIR, subject, session, 'func', f'{subject}_{session}_task-{task}_hemi-{hemi}_space-fsaverage6_bold.func.gii')
if not os.path.exists(surf_file):
    logging.error(f"Surface file {surf_file} does not exist.")

surf_data = surface.load_surf_data(surf_file)
sdata = np.array(surf_data, dtype=np.float64)

sdata = zscore(butterworth_highpass(sdata, tr, 0.01), axis=1)

# load wav
wav_file = os.path.join(STIMDIR, f'{task}.wav')
sr, audio = wavfile.read(wav_file)

audio_envelope = get_envelope(audio)[:, 0]

audio_envelope_avg = np.mean(audio_envelope[:len(audio_envelope) // int(sr * tr) * int(sr * tr)].reshape(-1, int(sr * tr)), axis=1)

zEnv = zscore(audio_envelope_avg)

sdata = sdata[:, 2:zEnv.shape[0]+2]

In [ ]:
# audio encoding model
n_reps = 10
alphas = np.logspace(-3, 3, 7)
nfolds = 5

n_delays = 5 # 7.5 seconds
xdata = make_delayed(zEnv[:, np.newaxis], np.arange(1, n_delays + 1))

encperf_all = np.zeros((n_reps, sdata.shape[0]))
for i_rep in range(n_reps):
    wt, corr, best_alphas, bscorrs, valinds, voxcorrs = ridge_cv(xdata, sdata.T, alphas, nfolds)
    encperf_all[i_rep, :] = voxcorrs

encperf = np.mean(encperf_all, axis=0)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4), subplot_kw={'projection': '3d'})

fl = plotting.plot_surf_stat_map(fsaverage.infl_left, encperf, hemi='left', view='lateral', colorbar=True, bg_map=fsaverage.sulc_left, cmap='seismic', vmax=0.5, vmin=-0.5, threshold=0.1, axes=axes[0], title='Treasure Island \n Envelope encoding performance (lateral)')
fm = plotting.plot_surf_stat_map(fsaverage.infl_left, encperf, hemi='left', view='medial', colorbar=True, bg_map=fsaverage.sulc_left, cmap='seismic', vmax=0.5, vmin=-0.5, threshold=0.1, axes=axes[1], title='Treasure Island \n Envelope encoding performance (medial)')